# Langchain Model Prompt and Parser

In this course we will learn how to use langchain.

![components of langchain](assets/components_langchain.png)

Setup the environment and especially set the model and API key

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

We now use a traditional approach to prompt engineering, i.e. without langchain

In [2]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message["content"]

In [3]:
get_completion("What is 1+1?")

'2'

In [4]:
customer_mail = """
Arr, I be fuming that me blender lid flew off and splattered me parrot across the room. I need your help matey."""

In [22]:
style = """American English in a nice and polite way"""

In [23]:
prompt = f"""translate the text that is delimeted in triple backticks into 
language and style that is {style} 
```{customer_mail}```
"""

In [24]:
print (prompt)


translate the text that is delimeted in triple backticks into 
language and style that is American English in a nice and polite way 
```
Arr, I be fuming that me blender lid flew off and splattered me parrot across the room. I need your help matey.```



In [25]:
print(get_completion(prompt))

I'm quite upset that my blender lid came off and splattered my parrot across the room. I need your help, my friend.


In [28]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.0)

In [46]:
template_string = """translate the text that is delimeted in triple backticks into
language and style that is {style} 
```{text}```"""

Nun nutzen wir langchain um diesen ganzen Prozess mit Zusammenbau der Prompts aus verschiedenen strings und die Übergabe an das Modell zu streamlinen.

## Langchain Prompttemplates

In [47]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

Die Promptemplates werden automatisch von langchain analysiert und die variablen Anteile werden als Variablenliste definiert, so dass man diese dann übergeben kann.

In [48]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='translate the text that is delimeted in triple backticks into\nlanguage and style that is {style} \n```{text}```', template_format='f-string', validate_template=True)

In [49]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [50]:
customer_style = """American English \
in a calm and respectful tone
"""

In [51]:
customer_mail = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
Das Prompttemplate kann nun leicht mit Datenbestückt und als message an das Chatmodell übergeben werden.

In [52]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_mail)

In [53]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [54]:
print(customer_messages[0])

content="translate the text that is delimeted in triple backticks into\nlanguage and style that is American English in a calm and respectful tone\n \n```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```" additional_kwargs={} example=False


In [55]:
customer_response = chat(customer_messages)

Aif diese Weise kann der Dialog zwischen Kunde und Servicemitarbeiter einfach gestaltet werden.

In [56]:
print(customer_response.content)

I must admit, I am quite frustrated that my blender lid unexpectedly flew off and caused a messy splatter of smoothie all over my kitchen walls. To add to my dismay, the warranty does not cover the expenses of cleaning up my kitchen. I kindly request your assistance at this moment, my friend.


In [57]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [58]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [59]:
customer_messages = prompt_template.format_messages(
                    style=service_style_pirate,
                    text=service_reply)

In [60]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

translate the text that is delimeted in triple backticks into
language and style that is a polite tone that speaks in English Pirate 
```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```


In [61]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, matey! Regrettably, the warranty be not coverin' the costs o' cleanin' yer galley, as 'tis yer own fault fer mishandlin' yer blender. Ye be forgettin' to secure the lid afore settin' the blender in motion. Aye, 'tis a tough break, indeed! Fare thee well, me heartie!


So why should we use langchain prompt-templates instead of f-strings? In practice prompts get pretty long pretty soon and a good way to standardize and systematize prompt engineering is helpful. Also we might to reuse prompts again again.. Especially we might want to build complex prompts from well proven simpler prompts as their constiutents (reuse).

Also there are standard prompts for typical tasks, like querying a database etc.

Also we can facilitate a certain form of prompts. For instance chain-of-thought prompting that uses the ReAct prompting framework will need a very special output style that is returend by the LLM and this means that this form has to be specified in detail by the prompt. A proven template is helpful.

![chain of thought prompting](assets/chain_of_thought_prompting.png)

## Parsers

suppose we want to extract json in a given form from customer reviews.

In [62]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [63]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [64]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [66]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [67]:
type(response.content)

str

In [68]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

Das LLM hat uns inhaltlich eine gute Struktur zurückgegeben, allerdings geben LLM nur Text also stings zurück. Wir haben kein json zurück erhalten, sondern einen String der die gleiche Information enthält. Hier setzen dann die Parser an.

In [69]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [70]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [71]:
print(response_schemas)

[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')]


In [72]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [73]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [74]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [75]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [76]:
response = chat(messages)
print(response.content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [77]:
output_dict = output_parser.parse(response.content)
output_dict
type(output_dict)
output_dict.get('delivery_days')

'2'